## Database

---


In [21]:
# PATHs

# The directory where the dune data is stored
ORI_DATA_PATH = './ori_data'

# The directory where the three databases are stored
DATABASE_PATH = './database'

# The directory where the scraped tweets data is stored
TWEET_PATH = './ori_data/tweets'


In [24]:
import time
import os
import json
import pandas as pd
import numpy as np
from datetime import date


In [10]:
today = str(date.today())

dune_bt_dict = json.load(
    open('{}/punkBought_{}.json'.format(ORI_DATA_PATH, today)))
dune_tf_dict = json.load(
    open('{}/punkTransfer_{}.json'.format(ORI_DATA_PATH, today)))


### 1. Transaction Database

- **tx_id**
- date_time
- from: trader_id
- yo: trader_id
- eth_price
- punk_id: punk_id


In [11]:
def index_trader(buyer, seller):
    trader = list(set(list(buyer.unique()) + list(seller.unique())))

    return {trader[i]: i for i in range(len(trader))}


In [12]:
def create_tx_db(dune_bt_dict, dune_tf_dict):
    temp_idx = 0
    # punkBought data frame
    bt_data = dict()
    for year in dune_bt_dict:
        data_list = dune_bt_dict[year]['data']['get_result_by_result_id']
        bt_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # punkTransfer data frame
    tf_data = dict()
    for year in dune_tf_dict:
        data_list = dune_tf_dict[year]['data']['get_result_by_result_id']
        tf_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # concatenate data frames
    tx_db = pd.concat([pd.DataFrame.from_dict(bt_data, orient='index'),
                       pd.DataFrame.from_dict(tf_data, orient='index')])

    # remove the wrong tx
    tx_db = tx_db[tx_db['eth_price'] < 10000]

    # set date_time
    tx_db['date_time'] = pd.to_datetime(tx_db['_date'] + ' ' + tx_db['_time'])

    # remove txs whose buyer is \x0000000000000000000000000000000000000000
    tx_db = tx_db[tx_db['buyer'] !=
                  '\\x0000000000000000000000000000000000000000']

    # reindex seller and buyer using trader_id
    trader_index_dict = index_trader(tx_db['buyer'], tx_db['seller'])
    tx_db['from'] = tx_db['seller'].apply(lambda x: trader_index_dict[x])
    tx_db['to'] = tx_db['buyer'].apply(lambda x: trader_index_dict[x])

    # sort by date_time
    tx_db.sort_values(by='date_time', inplace=True)

    # index transaction
    tx_db['tx_id'] = range(len(tx_db))
    tx_db = tx_db.loc[:, ['tx_id', 'date_time',
                          'from', 'to', 'eth_price', 'punk_id']]
    tx_db.set_index('tx_id', inplace=True)

    print('Total {} transactions'.format(len(tx_db)))
    print('Total {} unique traders'.format(len(trader_index_dict)))
    print('Total {} unique punk_id'.format(len(tx_db['punk_id'].unique())))

    return tx_db, trader_index_dict


tx_db, trader_index_dict = create_tx_db(dune_bt_dict, dune_tf_dict)
tx_db.to_csv('{}/tx_db.csv'.format(DATABASE_PATH), index=True)
print('Transaction database saved to {}/tx_db.csv'.format(DATABASE_PATH))
tx_db


Total 16824 transactions
Total 5912 unique traders
Total 6157 unique punk_id
Transaction database saved to ./database/tx_db.csv


,date_time,from,to,eth_price,punk_id
tx_id,,,,,
0,2017-06-23 21:06:32,5663,1382,0.01,3134
1,2017-06-23 21:06:45,5663,5349,0.04,5719
2,2017-06-23 21:06:53,1382,2438,0.10,5056
3,2017-06-23 22:06:08,5663,5349,0.06,5624
4,2017-06-23 22:06:12,5663,5349,0.03,6548
...,...,...,...,...,...
16819,2022-05-05 21:05:57,686,4633,70.00,3314
16820,2022-05-06 14:05:57,1055,5061,0.00,3095
16821,2022-05-06 17:05:48,2046,234,58.00,6995


### 2. CryptoPunk

- **punk_id**
- img_url
- type (Alien, Ape, Zombie, Female, Male)
- skin_tone
- attr_count
- attributes
- current_owner: trader_id
- avg_price


In [17]:

def get_avg_price(punk_id):
    tx_contains_punk_id = tx_db[tx_db['punk_id'] == punk_id]
    if tx_contains_punk_id.shape[0] == 0:
        return 0.0
    price_list = tx_contains_punk_id['eth_price'].tolist()
    avg_price = np.mean(price_list)
    return avg_price


def create_punk_db(CSV_PATH=ORI_DATA_PATH):
    punk_db = pd.read_csv('{}/{}'.format(CSV_PATH, 'punk_info.csv'))

    # rename columns
    punk_db.columns = ['punk_id', 'type', 'gender',
                       'skin_tone', 'attr_count', 'attributes']

    # strip type, gender, skin_tone
    punk_db['type'] = punk_db['type'].apply(lambda x: x.strip())
    punk_db['gender'] = punk_db['gender'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(
        lambda x: 'Non-human' if x == '' else x)

    # make attributes as list
    punk_db['attributes'] = punk_db['attributes'].apply(
        lambda x: [i.strip() for i in x.split('/')])

    # get punk avg price
    punk_db['avg_price'] = punk_db['punk_id'].apply(get_avg_price)

    # set punk image url
    punk_db['img_url'] = punk_db['punk_id'].apply(
        lambda x: 'https://www.larvalabs.com/cryptopunks/cryptopunk{}.png'.format(x))

    # set punk_id as index
    punk_db.set_index('punk_id', inplace=True)

    return punk_db


punk_db = create_punk_db(ORI_DATA_PATH)
punk_db.to_csv('{}/punk_db.csv'.format(DATABASE_PATH), index=True)
print('Cryptopunk database saved to {}/punk_db.csv'.format(DATABASE_PATH))
punk_db


Cryptopunk database saved to ./database/punk_db.csv


,type,gender,skin_tone,attr_count,attributes,avg_price,img_url
punk_id,,,,,,,
0,Human,Female,Medium,3,"[Green Eye Shadow, Earring, Blonde Bob]",0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
1,Human,Male,Dark,2,"[Smile, Mohawk]",31.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
2,Human,Female,Light,1,[Wild Hair],0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
3,Human,Male,Dark,3,"[Wild Hair, Nerd Glasses, Pipe]",0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
4,Human,Male,Medium,4,"[Big Shades, Wild Hair, Earring, Goat]",0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...
9995,Human,Female,Albino,2,"[Purple Eye Shadow, Straight Hair Dark]",0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
9996,Human,Male,Light,4,"[Cigarette, Earring, Crazy Hair, Smile]",0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
9997,Zombie,Male,Non-human,2,"[Front Beard, Cap Forward]",99.990000,https://www.larvalabs.com/cryptopunks/cryptopu...


### 3. Trader

- **trader_id**
- address
- tx_involved_count
- tx_involved: list of tx_id
- frequent_skin_tone
- frequent_gender


In [19]:

def find_most_frequent_skin_tone(tx_involved_list, tx_db):
    skin_tones = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        skin_tones.append(punk_db.loc[punk_id, 'skin_tone'])
    return max(set(skin_tones), key=skin_tones.count)


def find_most_frequent_gender(tx_involved_list, tx_db):
    genders = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        genders.append(punk_db.loc[punk_id, 'gender'])
    return max(set(genders), key=genders.count)


def create_trader_db(tx_db, trader_index_dict):
    trader_db = pd.DataFrame.from_dict(
        trader_index_dict, orient='index', columns=['trader_id'])

    # set address for each trader
    trader_db.reset_index(inplace=True)
    trader_db.rename(columns={'index': 'address'}, inplace=True)

    # set trader_id as index
    trader_db.sort_values(by='trader_id', inplace=True)

    # get the involved tx
    trader_db['tx_involved_count'] = trader_db['trader_id'].apply(
        lambda x: len(tx_db[tx_db['from'] == x]) + len(tx_db[tx_db['to'] == x]))
    trader_db['tx_involved'] = trader_db['trader_id'].apply(lambda x: sorted(list(tx_db[tx_db['from'] == x].index) +
                                                            list(tx_db[tx_db['to'] == x].index)))

    # get the skin tone that each trader most frequently trades with
    trader_db['frequent_skin_tone'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_skin_tone(x, tx_db))

    # get the gender that each trader most frequently trades with
    trader_db['frequent_gender'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_gender(x, tx_db))

    # set trader_id as index
    trader_db.set_index('trader_id', inplace=True)

    return trader_db


trader_db = create_trader_db(tx_db, trader_index_dict)
trader_db.to_csv('{}/trader_db.csv'.format(DATABASE_PATH), index=True)
print('Trader database saved to {}/trader_db.csv'.format(DATABASE_PATH))
trader_db


Trader database saved to ./database/trader_db.csv


,address,tx_involved_count,tx_involved,frequent_skin_tone,frequent_gender
trader_id,,,,,
0,\xc97c6698ac5eb041ee49b9acb6a7edfd373c4adb,1,[13235],Medium,Male
1,\x6bfe59e8feaddfbdbb97f7da8c6b4550a8bd4ffd,2,"[6786, 12831]",Light,Female
2,\xfc2a616d48a8681250aaaf590404e20812e96cfa,22,"[6636, 6658, 6667, 6677, 6687, 6702, 6716, 753...",Medium,Male
3,\x224ce899f5124ee40c1fdcb791383e628a4caae0,1,[9262],Medium,Male
4,\x69bb864c0ab4055bd19e9cbee35d1097727c8f84,1,[12263],Light,Male
...,...,...,...,...,...
5907,\x859ba94ff84f45af63aef746e5bccb57734f03c8,1,[14978],Medium,Male
5908,\x166149bca5fd1192208483ac2f8abd280fbb2561,1,[3565],Light,Male
5909,\xa4fb10666494d1d99ea305065a623401b48aceb8,1,[8667],Dark,Male


In [22]:
csv_list = list()
csv_counter = 0
for root, dirs, files in os.walk(TWEET_PATH):
    for file in files:
        if file.endswith(".csv"):
            temp_tweets_df = pd.read_csv(
                '{}/{}'.format(TWEET_PATH, file), lineterminator='\n')
            csv_list.append(temp_tweets_df)
            csv_counter += 1
            print('{} files read: {} ===== {} tweets'.format(
                csv_counter, file, len(temp_tweets_df)))

tweets_df = pd.concat(csv_list, axis=0, ignore_index=True)
tweets_df.drop(['Unnamed: 0'], axis=1, inplace=True)
tweets_df.sort_values(by=['date'], inplace=True)
tweets_df['id'] = tweets_df['id'].apply(lambda x: int(x))

tweets_df.drop(['retweetedTweet', 'mentionedUsers'], axis=1, inplace=True)

tweets_df = tweets_df.loc[:, ['id', 'date', 'content', 'url', 'username']]

tweets_df.to_csv('{}/tweets_db.csv'.format(DATABASE_PATH), index=False)

print('Tweets database saved to {}/tweets_db.csv'.format(DATABASE_PATH))
print('Total tweets: {}'.format(len(tweets_df)))

tweets_df.head()


1 files read: contain_nft_transparency.csv ===== 8794 tweets
2 files read: contain_cryptopunk_gender.csv ===== 34 tweets
3 files read: contain_cryptopunk_accountability.csv ===== 3 tweets
4 files read: contain_nft_skin_tone.csv ===== 198 tweets
5 files read: contain_cryptopunk_fairness.csv ===== 1 tweets
6 files read: contain_nft_gender.csv ===== 6456 tweets
7 files read: contain_cryptopunk_transparency.csv ===== 14 tweets
8 files read: contain_cryptopunk_skin_color.csv ===== 7 tweets
9 files read: contain_cryptopunk_ethnicity.csv ===== 0 tweets
10 files read: contain_nft_skin_color.csv ===== 389 tweets
11 files read: contain_nft_informed_consent.csv ===== 6 tweets
12 files read: contain_cryptopunk_skin_tone.csv ===== 5 tweets
13 files read: contain_nft_trust.csv ===== 54405 tweets
14 files read: contain_cryptopunk_ethic.csv ===== 3 tweets
15 files read: contain_nft_accountability.csv ===== 1561 tweets
16 files read: contain_nft_fairness.csv ===== 1706 tweets
17 files read: contain_nft

,id,date,content,url,username
70311,1254597572,2009-02-26 18:08:50+00:00,@wkriesel Thanks for #education RT--love you...,https://twitter.com/MayaFrost_NFT/status/12545...,MayaFrost_NFT
70310,1389182849,2009-03-25 17:31:52+00:00,A World Apart: Lessons Learned from Successfu...,https://twitter.com/MayaFrost_NFT/status/13891...,MayaFrost_NFT
81531,13964319067,2010-05-14 08:14:27+00:00,EU privacy watchdogs say Facebook changes 'una...,https://twitter.com/MichaelM_NFT/status/139643...,MichaelM_NFT
81530,14245622932,2010-05-18 19:28:31+00:00,New apps restore Facebook privacy settings - h...,https://twitter.com/MichaelM_NFT/status/142456...,MichaelM_NFT
81529,14245676874,2010-05-18 19:29:44+00:00,"MySpace's New Privacy Pitch: Too Little, Too L...",https://twitter.com/MichaelM_NFT/status/142456...,MichaelM_NFT


## Data Engineering for Visualization

---

Visualization 1: `vis1.ipynb`

Visualization 2: `vis2.ipynb`

Visualization 3: `vis3.ipynb`

Visualization 4: `vis4.ipynb`

Visualization 5: `vis5.ipynb`
